In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import re
from pathlib import Path

import pandas as pd
import xarray as xr
import pandas_indexing.accessors
from aneris.harmonize import Harmonizer
from aneris.downscaling import Downscaler
from aneris.grid import Gridder
from pandas import DataFrame
from pandas_indexing import isin, semijoin

from concordia import VariableDefinitions, RegionMapping

In [4]:
# Potentially better gridding performance??
# from dask.distributed import Client
# client = Client()

# Read model and historic data including overrides

In [4]:
base_path = Path(
    "/Users/coroa/Library/CloudStorage/OneDrive-SharedLibraries-IIASA/RESCUE - WP 1/data"
)
data_path = Path("../data")
base_year = 2015 # in which year scenario data should be harmonized to historical data

## Variable definition files

The variable definition file is a CSV or yaml file that needs to contain the `variable`-name, its `sector`, `gas` components and whether it is expected `global` (or regional instead).

Here we generate one based on the cmip6 historical data we have that could be used as a basis but we would want to finetune this by hand.

In [5]:
variabledefs = VariableDefinitions.from_csv(data_path / "variabledefs.csv")
variabledefs.data.head()

,,,unit,global
variable,gas,sector,,
CEDS+|9+ Sectors|Emissions|BC|Agricultural Waste Burning|Unharmonized,BC,Agricultural Waste Burning,Mt BC/yr,False
CEDS+|9+ Sectors|Emissions|BC|Agriculture|Unharmonized,BC,Agriculture,Mt BC/yr,False
CEDS+|9+ Sectors|Emissions|BC|Aircraft|Unharmonized,BC,Aircraft,Mt BC/yr,True
CEDS+|9+ Sectors|Emissions|BC|Energy Sector|Unharmonized,BC,Energy Sector,Mt BC/yr,False
CEDS+|9+ Sectors|Emissions|BC|Forest Burning|Unharmonized,BC,Forest Burning,Mt BC/yr,False


## RegionMapping helps reading in a region definition file

In [6]:
regionmapping = RegionMapping.from_regiondef(
    base_path / "historical/cmip6/remind_region_mapping.csv"
)
regionmapping.data.head()

country
afg    OAS
alb    ROW
dza    MEA
ago    AFR
atg    LAM
Name: region, dtype: object

## Model and historic data read in

Can be read in and prepared using `read_iamc` or the `variabledefs`

In [7]:
hist = pd.read_csv(
    base_path / "historical/cmip6/history.csv", index_col=list(range(5))
).pipe(variabledefs.load_data, levels=["region", "gas", "sector", "unit"])
hist.head()

,,,year,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,...,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
region,gas,sector,unit,,,,,,,,,,,,,,,,,,,,,
World,BC,Agricultural Waste Burning,Mt BC/yr,1.995380e-01,2.214554e-01,1.835703e-01,2.010172e-01,2.187860e-01,2.024483e-01,1.997409e-01,2.114301e-01,2.483196e-01,1.790196e-01,...,0.218279,0.189355,0.228403,1.871301e-01,0.180565,0.160563,1.586978e-01,0.145540,0.184817,1.873268e-01
abw,BC,Agricultural Waste Burning,Mt BC/yr,7.300000e-09,1.130000e-08,7.810000e-09,7.080000e-09,7.930000e-09,5.540000e-09,7.360000e-09,9.030000e-09,8.720000e-09,7.080000e-09,...,0.000000,0.000000,0.000000,2.050000e-08,0.000000,0.000000,8.710000e-08,0.000000,0.000000,1.076000e-08
afg,BC,Agricultural Waste Burning,Mt BC/yr,2.952110e-06,2.986174e-06,2.886556e-06,2.812248e-06,2.820738e-06,2.847859e-06,2.718585e-06,1.461890e-06,6.983582e-06,2.681442e-06,...,0.000004,0.000002,0.000003,1.284763e-06,0.000006,0.000003,7.505290e-07,0.000002,0.000001,2.601792e-06
ago,BC,Agricultural Waste Burning,Mt BC/yr,3.220589e-03,3.346063e-03,3.302260e-03,3.207212e-03,3.301694e-03,3.203143e-03,3.231582e-03,2.302201e-03,3.175263e-03,2.642421e-03,...,0.003820,0.003154,0.002417,2.512730e-03,0.002800,0.002728,2.775136e-03,0.002818,0.002689,3.017703e-03
alb,BC,Agricultural Waste Burning,Mt BC/yr,1.208894e-04,1.215974e-04,1.223055e-04,1.230135e-04,1.237215e-04,1.244295e-04,1.251376e-04,5.810621e-05,8.705932e-05,1.060884e-04,...,0.000015,0.000548,0.000066,3.159511e-05,0.000050,0.000193,2.585209e-04,0.000013,0.000015,1.233517e-04


In [8]:
model = (
    pd.read_excel(
        base_path / "iam_files/cmip6/REMIND-MAGPIE_SSP5-34-OS/ssp5-34-os.xlsx",
        sheet_name="data",
        index_col=list(range(5)),
    )
    .rename(index={"Mt CO2-equiv/yr": "Mt CO2-eq/yr"}, level="Unit")
    .pipe(
        variabledefs.load_data,
        levels=["model", "scenario", "region", "gas", "sector", "unit"],
        ignore_missing=True,
    )
)
model.head()

year                                                                           2005   
model         scenario       region gas sector                     unit               
REMIND-MAGPIE SSP5-34-OS-V27 AFR    BC  Agricultural Waste Burning Mt BC/yr  0.0439  \
                             CHN    BC  Agricultural Waste Burning Mt BC/yr  0.1055   
                             EUR    BC  Agricultural Waste Burning Mt BC/yr  0.0387   
                             IND    BC  Agricultural Waste Burning Mt BC/yr  0.0817   
                             JPN    BC  Agricultural Waste Burning Mt BC/yr  0.0000   

year                                                                           2010   
model         scenario       region gas sector                     unit               
REMIND-MAGPIE SSP5-34-OS-V27 AFR    BC  Agricultural Waste Burning Mt BC/yr  0.0633  \
                             CHN    BC  Agricultural Waste Burning Mt BC/yr  0.1164   
                             EUR    BC  Agricultural Waste Burning Mt BC/yr  0.0384   
                             IND    BC  Agricultural Waste Burning Mt BC/yr  0.0841   
                             JPN    BC  Agricultural Waste Burning Mt BC/yr  0.0000   

year                                                                           2015   
model         scenario       region gas sector                     unit               
REMIND-MAGPIE SSP5-34-OS-V27 AFR    BC  Agricultural Waste Burning Mt BC/yr  0.0799  \
                             CHN    BC  Agricultural Waste Burning Mt BC/yr  0.1243   
                             EUR    BC  Agricultural Waste Burning Mt BC/yr  0.0383   
                             IND    BC  Agricultural Waste Burning Mt BC/yr  0.0892   
                             JPN    BC  Agricultural Waste Burning Mt BC/yr  0.0000   

year                                                                           2020   
model         scenario       region gas sector                     unit               
REMIND-MAGPIE SSP5-34-OS-V27 AFR    BC  Agricultural Waste Burning Mt BC/yr  0.0966  \
                             CHN    BC  Agricultural Waste Burning Mt BC/yr  0.1322   
                             EUR    BC  Agricultural Waste Burning Mt BC/yr  0.0383   
                             IND    BC  Agricultural Waste Burning Mt BC/yr  0.0943   
                             JPN    BC  Agricultural Waste Burning Mt BC/yr  0.0000   

year                                                                           2030   
model         scenario       region gas sector                     unit               
REMIND-MAGPIE SSP5-34-OS-V27 AFR    BC  Agricultural Waste Burning Mt BC/yr  0.1279  \
                             CHN    BC  Agricultural Waste Burning Mt BC/yr  0.1400   
                             EUR    BC  Agricultural Waste Burning Mt BC/yr  0.0369   
                             IND    BC  Agricultural Waste Burning Mt BC/yr  0.1054   
                             JPN    BC  Agricultural Waste Burning Mt BC/yr  0.0000   

year                                                                           2040   
model         scenario       region gas sector                     unit               
REMIND-MAGPIE SSP5-34-OS-V27 AFR    BC  Agricultural Waste Burning Mt BC/yr  0.1568  \
                             CHN    BC  Agricultural Waste Burning Mt BC/yr  0.1509   
                             EUR    BC  Agricultural Waste Burning Mt BC/yr  0.0357   
                             IND    BC  Agricultural Waste Burning Mt BC/yr  0.1061   
                             JPN    BC  Agricultural Waste Burning Mt BC/yr  0.0000   

year                                                                           2050   
model         scenario       region gas sector                     unit               
REMIND-MAGPIE SSP5-34-OS-V27 AFR    BC  Agricultural Waste Burning Mt BC/yr  0.1768  \
                             CHN    BC  Agricultural Waste Burning Mt BC/yr  0.

In [9]:
harm_overrides = (
    pd.read_excel(
        base_path / "iam_files/cmip6/REMIND-MAGPIE_SSP5-34-OS/ssp5-34-os.xlsx",
        sheet_name="harmonization",
        index_col=list(range(4)),
        usecols=list(range(5)),
    )
    .rename_axis(index=str.lower)
    .rename(columns=str.lower)
    .pipe(
        variabledefs.load_data,
        ignore_missing=True,
        levels=["region", "gas", "sector"],
        timeseries=False,
    )
    .method
)

harm_overrides.head()

region  gas  sector        
CHN     BC   Energy Sector     reduce_offset_2150
AFR     BC   Forest Burning        constant_ratio
EUR     BC   Forest Burning        constant_ratio
ROW     BC   Forest Burning        constant_ratio
USA     BC   Forest Burning        constant_ratio
Name: method, dtype: object

In [10]:
model.head()

year                                                                           2005   
model         scenario       region gas sector                     unit               
REMIND-MAGPIE SSP5-34-OS-V27 AFR    BC  Agricultural Waste Burning Mt BC/yr  0.0439  \
                             CHN    BC  Agricultural Waste Burning Mt BC/yr  0.1055   
                             EUR    BC  Agricultural Waste Burning Mt BC/yr  0.0387   
                             IND    BC  Agricultural Waste Burning Mt BC/yr  0.0817   
                             JPN    BC  Agricultural Waste Burning Mt BC/yr  0.0000   

year                                                                           2010   
model         scenario       region gas sector                     unit               
REMIND-MAGPIE SSP5-34-OS-V27 AFR    BC  Agricultural Waste Burning Mt BC/yr  0.0633  \
                             CHN    BC  Agricultural Waste Burning Mt BC/yr  0.1164   
                             EUR    BC  Agricultural Waste Burning Mt BC/yr  0.0384   
                             IND    BC  Agricultural Waste Burning Mt BC/yr  0.0841   
                             JPN    BC  Agricultural Waste Burning Mt BC/yr  0.0000   

year                                                                           2015   
model         scenario       region gas sector                     unit               
REMIND-MAGPIE SSP5-34-OS-V27 AFR    BC  Agricultural Waste Burning Mt BC/yr  0.0799  \
                             CHN    BC  Agricultural Waste Burning Mt BC/yr  0.1243   
                             EUR    BC  Agricultural Waste Burning Mt BC/yr  0.0383   
                             IND    BC  Agricultural Waste Burning Mt BC/yr  0.0892   
                             JPN    BC  Agricultural Waste Burning Mt BC/yr  0.0000   

year                                                                           2020   
model         scenario       region gas sector                     unit               
REMIND-MAGPIE SSP5-34-OS-V27 AFR    BC  Agricultural Waste Burning Mt BC/yr  0.0966  \
                             CHN    BC  Agricultural Waste Burning Mt BC/yr  0.1322   
                             EUR    BC  Agricultural Waste Burning Mt BC/yr  0.0383   
                             IND    BC  Agricultural Waste Burning Mt BC/yr  0.0943   
                             JPN    BC  Agricultural Waste Burning Mt BC/yr  0.0000   

year                                                                           2030   
model         scenario       region gas sector                     unit               
REMIND-MAGPIE SSP5-34-OS-V27 AFR    BC  Agricultural Waste Burning Mt BC/yr  0.1279  \
                             CHN    BC  Agricultural Waste Burning Mt BC/yr  0.1400   
                             EUR    BC  Agricultural Waste Burning Mt BC/yr  0.0369   
                             IND    BC  Agricultural Waste Burning Mt BC/yr  0.1054   
                             JPN    BC  Agricultural Waste Burning Mt BC/yr  0.0000   

year                                                                           2040   
model         scenario       region gas sector                     unit               
REMIND-MAGPIE SSP5-34-OS-V27 AFR    BC  Agricultural Waste Burning Mt BC/yr  0.1568  \
                             CHN    BC  Agricultural Waste Burning Mt BC/yr  0.1509   
                             EUR    BC  Agricultural Waste Burning Mt BC/yr  0.0357   
                             IND    BC  Agricultural Waste Burning Mt BC/yr  0.1061   
                             JPN    BC  Agricultural Waste Burning Mt BC/yr  0.0000   

year                                                                           2050   
model         scenario       region gas sector                     unit               
REMIND-MAGPIE SSP5-34-OS-V27 AFR    BC  Agricultural Waste Burning Mt BC/yr  0.1768  \
                             CHN    BC  Agricultural Waste Burning Mt BC/yr  0.

# Harmonization

## Preparation of input data

In [11]:
hist_agg = pd.concat(
    [
        hist.idx.semijoin(variabledefs.index_regional, how="inner").pipe(
            regionmapping.aggregate, level="region"
        ),
        hist.idx.semijoin(variabledefs.index_global, how="inner").loc[
            isin(region="World")
        ],
    ]
)

In [12]:
model_agg = pd.concat(
    [
        model.idx.semijoin(variabledefs.index_regional, how="inner").loc[
            isin(region=regionmapping.data.unique())
        ],
        model.idx.semijoin(variabledefs.index_global, how="inner").loc[
            isin(region="World")
        ],
    ]
)

## Harmonize all model, scenarios combinations

In [13]:
harmonized = []
for m, s in model.index.idx.project(["model", "scenario"]).unique():
    scen = model_agg.loc[isin(model=m, scenario=s)].droplevel(["model", "scenario"])
    h = Harmonizer(
        scen, hist_agg.idx.semijoin(scen.index, how="right"), harm_idx=scen.index.names
    )
    result = h.harmonize(year=base_year, overrides=harm_overrides)
    harmonized.append(result.idx.assign(model=m, scenario=s))
harmonized = pd.concat(harmonized)

# TODO harmonization casts columns to str!!
harmonized = harmonized.rename(columns=int)
harmonized.head()

/Users/coroa/repos/aneris/src/aneris/methods.py:389: RuntimeWarning: invalid value encountered in scalar divide
  return np.abs(np.std(x) / np.mean(x))
INFO:root:Harmonizing with reduce_offset_2150_cov
INFO:root:Harmonizing with model_zero
INFO:root:Harmonizing with hist_zero
INFO:root:Harmonizing with constant_ratio
INFO:root:Harmonizing with reduce_offset_2150
INFO:root:Harmonizing with reduce_ratio_2080
INFO:root:Harmonizing with constant_offset
INFO:root:Harmonizing with reduce_offset_2100


year                                                                             2015   
gas sector                     region unit     model         scenario                   
BC  Agricultural Waste Burning AFR    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.058867  \
                               CHN    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.013035   
                               EUR    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.004719   
                               IND    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.005197   
                               JPN    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.000437   

year                                                                             2020   
gas sector                     region unit     model         scenario                   
BC  Agricultural Waste Burning AFR    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.076346  \
                               CHN    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.025056   
                               EUR    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.005963   
                               IND    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.013408   
                               JPN    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.000437   

year                                                                             2030   
gas sector                     region unit     model         scenario                   
BC  Agricultural Waste Burning AFR    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.109204  \
                               CHN    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.041098   
                               EUR    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.007051   
                               IND    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.030730   
                               JPN    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.000437   

year                                                                             2040   
gas sector                     region unit     model         scenario                   
BC  Agricultural Waste Burning AFR    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.139662  \
                               CHN    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.060239   
                               EUR    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.008338   
                               IND    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.037653   
                               JPN    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.000437   

year                                                                             2050   
gas sector                     region unit     model         scenario                   
BC  Agricultural Waste Burning AFR    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.161220  \
                               CHN    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.094981   
                               EUR    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.009125   
                               IND    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.040575   
                               JPN    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.000437   

year                                                                             2060   
gas sector                     region unit     model         scenario                   
BC  Agricultural Waste Burning AFR    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.146278  \
                               CHN    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.085123   
                               EUR    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.011613   
                               IND    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.047498   
                               JPN    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.000437   

year                                                                             2070   
gas sector                     region unit     model         scenario                   
BC  Agricultural Waste Burning AFR    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0

# Downscaling

## Prepare GDP proxy

Read in different GDP scenarios for SSP1 to SSP5 from SSP DB.

In [14]:
gdp = (
    pd.read_csv(
        base_path / "historical" / "SspDb_country_data_2013-06-12.csv",
        index_col=list(range(5)),
    )
    .rename_axis(index=str.lower)
    .loc[
        isin(
            model="OECD Env-Growth",
            scenario=[f"SSP{n+1}_v9_130325" for n in range(5)],
            variable="GDP|PPP",
        )
    ]
    .dropna(how="all", axis=1)
    .rename_axis(index={"scenario": "ssp", "region": "country"})
    .rename(index=str.lower, level="country")
    .rename(columns=int)
    .idx.project(["ssp", "country"])
)
gdp.head()

2000    2005     2010     2015      2020      2025   
ssp            country                                                         
SSP1_v9_130325 bhs       9.179   9.964    9.647   10.852    12.364    13.913  \
               bih      18.823  23.975   28.065   30.809    37.787    47.037   
               blr      58.129  83.492  118.671  145.259   176.778   211.172   
               blz       1.404   1.825    2.061    2.266     2.592     3.141   
               twn       0.000   0.000  742.813  889.439  1073.988  1208.735   

                            2030      2035      2040      2045  ...      2055   
ssp            country                                          ...             
SSP1_v9_130325 bhs        15.577    17.400    19.244    21.071  ...    24.061  \
               bih        57.740    69.273    80.304    89.614  ...   101.533   
               blr       250.038   292.074   330.894   362.482  ...   404.928   
               blz         3.903     4.861     5.977     7.218  ...     9.869   
               twn      1321.155  1424.861  1513.084  1570.705  ...  1587.286   

                            2060      2065      2070      2075      2080   
ssp            country                                                     
SSP1_v9_130325 bhs        25.282    26.374    27.331    27.979    28.462  \
               bih       105.790   108.039   109.080   108.896   107.453   
               blr       424.762   443.176   455.985   461.441   462.309   
               blz        11.223    12.574    13.881    15.116    16.320   
               twn      1574.961  1608.617  1631.834  1638.589  1633.208   

                            2085      2090      2095      2100  
ssp            country                                          
SSP1_v9_130325 bhs        28.752    28.843    28.773    28.526  
               bih       105.156   102.274    99.093    95.538  
               blr       461.472   459.908   456.940   451.145  
               blz        17.463    18.516    19.481    20.309  
               twn      1610.471  1576.924  1534.288  1482.390  

[5 rows x 21 columns]

Determine likely SSP for each harmonized pathway from scenario string and create proxy data aligned with pathways

In [15]:
SSP_per_pathway = (
    harmonized.index.idx.project(["model", "scenario"])
    .unique()
    .to_frame()
    .scenario.str.extract("(SSP[1-5])")[0]
    .fillna("SSP2")
)
gdp = semijoin(
    gdp,
    SSP_per_pathway.index.idx.assign(ssp=SSP_per_pathway + "_v9_130325"),
    how="right",
).idx.project(["model", "scenario", "country"])
gdp.head()

2000    2005     2010     2015   
model         scenario       country                                     
REMIND-MAGPIE SSP5-34-OS-V27 bhs       9.179   9.964    9.647   10.887  \
                             bih      18.823  23.975   28.065   30.799   
                             blr      58.129  83.492  118.671  145.147   
                             blz       1.404   1.825    2.061    2.261   
                             twn       0.000   0.000  742.813  889.439   

                                          2020      2025      2030      2035   
model         scenario       country                                           
REMIND-MAGPIE SSP5-34-OS-V27 bhs        12.498    14.367    16.684    19.453  \
                             bih        37.978    48.557    61.832    76.844   
                             blr       177.168   216.865   266.159   322.456   
                             blz         2.574     3.138     3.989     5.121   
                             twn      1084.599  1261.959  1440.954  1627.208   

                                          2040      2045  ...      2055   
model         scenario       country                      ...             
REMIND-MAGPIE SSP5-34-OS-V27 bhs        22.427    25.551  ...    31.468  \
                             bih        91.632   104.935  ...   125.297   
                             blr       376.506   424.503  ...   503.192   
                             blz         6.434     7.884  ...    10.960   
                             twn      1806.415  1960.612  ...  2162.757   

                                          2060      2065      2070      2075   
model         scenario       country                                           
REMIND-MAGPIE SSP5-34-OS-V27 bhs        34.292    37.051    39.711    42.017  \
                             bih       134.230   141.058   146.504   150.487   
                             blr       544.293   585.440   620.998   648.367   
                             blz        12.552    14.185    15.797    17.376   
                             twn      2241.038  2384.591  2510.562  2613.281   

                                          2080      2085      2090      2095   
model         scenario       country                                           
REMIND-MAGPIE SSP5-34-OS-V27 bhs        44.129    46.010    47.633    49.027  \
                             bih       152.868   154.086   154.434   154.265   
                             blr       670.707   691.504   711.865   730.512   
                             blz        19.013    20.667    22.332    24.057   
                             twn      2703.523  2767.277  2813.176  2841.165   

                                          2100  
model         scenario       country            
REMIND-MAGPIE SSP5-34-OS-V27 bhs        50.126  
                             bih       153.375  
                             blr       744.978  
                             blz        25.806  
                             twn      2845.093  

[5 rows x 21 columns]

In [16]:
downscaler = Downscaler(
    harmonized.idx.semijoin(variabledefs.index_regional, how="inner"),
    hist.idx.semijoin(variabledefs.index_regional, how="inner").rename_axis(
        index={"region": "country"}
    ),
    base_year,
    regionmapping.data.drop(["srb (kosovo)", "ssd"]),
    gdp=gdp,
)
results = downscaler.downscale()

/Users/coroa/repos/aneris/src/aneris/methods.py:389: RuntimeWarning: invalid value encountered in scalar divide
  return np.abs(np.std(x) / np.mean(x))
/Users/coroa/repos/aneris/src/aneris/downscaling/intensity_convergence.py:328: RuntimeWarning: invalid value encountered in multiply
  (


In [17]:
downscaler.methods().value_counts()

/Users/coroa/repos/aneris/src/aneris/methods.py:389: RuntimeWarning: invalid value encountered in scalar divide
  return np.abs(np.std(x) / np.mean(x))


method
ipat_2100_gdp        781
proxy_gdp            132
base_year_pattern     33
Name: count, dtype: int64

In [18]:
results

year                                                                                                  2015   
gas    sector                              region unit      model         scenario       country             
CH4    Agriculture                         AFR    Mt CH4/yr REMIND-MAGPIE SSP5-34-OS-V27 ago      0.220823  \
                                                                                         bdi      0.036465   
                                                                                         ben      0.095516   
                                                                                         bfa      0.511754   
                                                                                         bwa      0.105129   
...                                                                                                    ...   
Sulfur Agriculture                         USA    Mt SO2/yr REMIND-MAGPIE SSP5-34-OS-V27 usa      0.000000   
       Solvents Production and Application USA    Mt SO2/yr REMIND-MAGPIE SSP5-34-OS-V27 pri      0.000000   
                                                                                         usa      0.000000   
VOC    Agriculture                         USA    Mt VOC/yr REMIND-MAGPIE SSP5-34-OS-V27 pri      0.000000   
                                                                                         usa      0.000000   

year                                                                                                  2020   
gas    sector                              region unit      model         scenario       country             
CH4    Agriculture                         AFR    Mt CH4/yr REMIND-MAGPIE SSP5-34-OS-V27 ago      0.279381  \
                                                                                         bdi      0.046135   
                                                                                         ben      0.120845   
                                                                                         bfa      0.647461   
                                                                                         bwa      0.133007   
...                                                                                                    ...   
Sulfur Agriculture                         USA    Mt SO2/yr REMIND-MAGPIE SSP5-34-OS-V27 usa      0.000000   
       Solvents Production and Application USA    Mt SO2/yr REMIND-MAGPIE SSP5-34-OS-V27 pri      0.000000   
                                                                                         usa      0.000000   
VOC    Agriculture                         USA    Mt VOC/yr REMIND-MAGPIE SSP5-34-OS-V27 pri      0.000000   
                                                                                         usa      0.000000   

year                                                                                                  2030   
gas    sector                              region unit      model         scenario       country             
CH4    Agriculture                         AFR    Mt CH4/yr REMIND-MAGPIE SSP5-34-OS-V27 ago      0.515045  \
                                                                                         bdi      0.085051   
                                                                                         ben      0.222780   
                                                                                         bfa      1.193611   
                                                                                         bwa      0.245201   
...                                                                                                    ...   
Sulfur Agriculture                         USA    Mt SO2/yr REMIND-MAGPIE SSP5-34-OS-V27 usa      0.000000   
       Solvents Production and Application USA    Mt SO2/yr REMIND-MAGPIE SSP5-34-OS-V27 pri      0.000000   
                                      

# Gridding

In [19]:
idxr = xr.open_dataarray(base_path / "gridding_process_files" / "iso_mask.nc", chunks={"iso": 20}).rename({"iso": "country"})

In [20]:
proxy_dir = base_path / "gridding_process_files" / "proxy_rasters"
proxy_cfg = pd.concat(
    [
        # DataFrame(
        #     {
        #         "path": proxy_dir.glob("aircraft_*.nc"),
        #         "name": "em-AIR-anthro",
        #         "separate_shares": False,
        #     }
        # ),
        DataFrame(
            {
                "path": proxy_dir.glob("anthro_*.nc"),
                "name": "em-anthro",
                "separate_shares": False,
            }
        ),
        # DataFrame(
        #     {
        #         "path": proxy_dir.glob("openburning_*.nc"),
        #         "name": "em-openburning",
        #         "separate_shares": True,
        #     }
        # ),
        # unsure where shipping is used??
        # DataFrame({"path": proxy_dir.glob("shipping_*.nc"), "name": ..., "template": ..., "separate_shares": False})
    ]
).assign(
    name=lambda df: df.path.map(lambda p: p.stem.split("_")[1]) + "-" + df.name,
    template="{name}_emissions_{model}-{scenario}_201501-210012",
)
proxy_cfg.head()

,path,name,separate_shares,template
0,/Users/coroa/Library/CloudStorage/OneDrive-Sha...,OC-em-anthro,False,{name}_emissions_{model}-{scenario}_201501-210012
1,/Users/coroa/Library/CloudStorage/OneDrive-Sha...,CO-em-anthro,False,{name}_emissions_{model}-{scenario}_201501-210012
2,/Users/coroa/Library/CloudStorage/OneDrive-Sha...,NH3-em-anthro,False,{name}_emissions_{model}-{scenario}_201501-210012
3,/Users/coroa/Library/CloudStorage/OneDrive-Sha...,NOx-em-anthro,False,{name}_emissions_{model}-{scenario}_201501-210012
4,/Users/coroa/Library/CloudStorage/OneDrive-Sha...,CO2-em-anthro,False,{name}_emissions_{model}-{scenario}_201501-210012


In [21]:
sector_mapping = {
    "AIR": "Aircraft",
    "SHP": "International Shipping",
    "AWB": "Agricultural Waste Burning",
    "AGR": "Agriculture",
    "ENE": "Energy Sector",
    "FRTB": "Forest Burning",
    "GRSB": "Grassland Burning",
    "IND": "Industrial Sector",
    "PEAT": "Peat Burning",
    "RCO": "Residential Commercial Other",
    "SLV": "Solvents Production and Application",
    "TRA": "Transportation Sector",
    "WST": "Waste",
}

In [42]:
kg_per_mt = 1e9
s_per_yr = 365 * 24 * 60 * 60
results = (
    results.rename(index=lambda s: re.sub("Mt (.*)/yr", r"kg \1/s", s), level="unit")
    * kg_per_mt
    / s_per_yr
)

In [43]:
results = results.droplevel("region")
results.head()

year                                                                2015   
gas sector      unit     model         scenario       country              
CH4 Agriculture kg CH4/s REMIND-MAGPIE SSP5-34-OS-V27 ago       7.002247  \
                                                      bdi       1.156298   
                                                      ben       3.028788   
                                                      bfa      16.227619   
                                                      bwa       3.333608   

year                                                                2020   
gas sector      unit     model         scenario       country              
CH4 Agriculture kg CH4/s REMIND-MAGPIE SSP5-34-OS-V27 ago       8.859105  \
                                                      bdi       1.462925   
                                                      ben       3.831963   
                                                      bfa      20.530863   
                                                      bwa       4.217615   

year                                                                2030   
gas sector      unit     model         scenario       country              
CH4 Agriculture kg CH4/s REMIND-MAGPIE SSP5-34-OS-V27 ago      16.331986  \
                                                      bdi       2.696940   
                                                      ben       7.064321   
                                                      bfa      37.849168   
                                                      bwa       7.775281   

year                                                                2040   
gas sector      unit     model         scenario       country              
CH4 Agriculture kg CH4/s REMIND-MAGPIE SSP5-34-OS-V27 ago      27.781679  \
                                                      bdi       4.587656   
                                                      ben      12.016831   
                                                      bfa      64.383687   
                                                      bwa      13.226216   

year                                                                2050   
gas sector      unit     model         scenario       country              
CH4 Agriculture kg CH4/s REMIND-MAGPIE SSP5-34-OS-V27 ago      24.593980  \
                                                      bdi       4.061263   
                                                      ben      10.638007   
                                                      bfa      56.996234   
                                                      bwa      11.708626   

year                                                                2060   
gas sector      unit     model         scenario       country              
CH4 Agriculture kg CH4/s REMIND-MAGPIE SSP5-34-OS-V27 ago      12.604726  \
                                                      bdi       2.081449   
                                                      ben       5.452113   
                                                      bfa      29.211290   
                                                      bwa       6.000819   

year                                                                2070   
gas sector      unit     model         scenario       country              
CH4 Agriculture kg CH4/s REMIND-MAGPIE SSP5-34-OS-V27 ago       5.649102  \
                                                      bdi       0.932850   
                                                      ben       2.443492   
                                                      bfa      13.091722   
                                                      bwa       2.689407   

year                                                                2080   
gas sector      unit     model         scenario       country              
CH4 Agriculture kg CH4/s REMIND-MAGPIE SSP5-34-OS-V27 ago       4.963229  \
                                         

In [45]:
gridder = Gridder(results, idxr, proxy_cfg, index_mappings=dict(sector=sector_mapping), output_dir="../results")

In [50]:
gridder.grid(skip_check=True)

INFO:root:Collecting tasks for proxy OC-em-anthro
/Users/coroa/.local/conda/envs/concordia/lib/python3.11/site-packages/dask/array/core.py:4832: PerformanceWarning: Increasing number of chunks by factor of 12
  result = blockwise(
INFO:root:Adding tasks for {'unit': 'kg OC/s', 'model': 'REMIND-MAGPIE', 'scenario': 'SSP5-34-OS-V27'}
INFO:root:Writing to ../results/OC-em-anthro_emissions_REMIND-MAGPIE-SSP5-34-OS-V27_201501-210012.nc
INFO:root:Adding tasks for {'unit': 'kg OC/s', 'model': 'IMAGE', 'scenario': 'SSP2'}
INFO:root:Writing to ../results/OC-em-anthro_emissions_IMAGE-SSP2_201501-210012.nc


[                                        ] | 0% Completed | 8.98 s ms

/Users/coroa/.local/conda/envs/concordia/lib/python3.11/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))


[###################################     ] | 87% Completed | 11m 13ss